##Topic Modelling in Urdu by using LSA model
In this notebook we are going to implement LSA (Latent Semantic Analysis) model for urdu language.

#Mounting Google Drive with Collaboratory
If your dataset is present in google drive then do this step and join your google drive with collaboratory.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


##Importing Dependencicies
We will import the required libraries which we are going to work in this notebook. Some of the libraries will be imported later as we build the model.

In [ ]:
import pandas as pd
import numpy as np
import re

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


##Peeking the data

The we are going to reda csv file contaning more than 1 Million news from different sources related to different categories.

df.head() will print the firt 5 rows of the dataset.

In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/Dataset(Colab)/UrduTopicModelling/urdu-news-dataset-1M.csv',encoding='utf-8')
df.head()

,Index,Headline,News Text,Category,Date,URL,Source,News length
0,0,عالمی بینک عسکریت پسندی سے متاثرہ خاندانوں کی ...,اسلام باد عالمی بینک خیبرپختونخوا کے قبائلی اض...,Business & Economics,12/6/2020,https://www.dawnnews.tv/news/1148499/,Dawn News,1854.0
1,1,مالی سال 2020 ریٹرن فائل کرنے والوں کی تعداد م...,اسلام باد فیڈرل بورڈ ریونیو ایف بی نے دسمبر کی...,Business & Economics,12/6/2020,https://www.dawnnews.tv/news/1148498/,Dawn News,2016.0
2,2,جاپان کو سندھ کے خصوصی اقتصادی زون میں سرمایہ ...,اسلام باد بورڈ انویسٹمنٹ بی او ئی کے چیئرمین ع...,Business & Economics,12/5/2020,https://www.dawnnews.tv/news/1148433/,Dawn News,2195.0
3,3,برامدات 767 فیصد بڑھ کر ارب 16 کروڑ ڈالر سے زائد,اسلام اباد پاکستان میں ماہ نومبر میں مسلسل تیس...,Business & Economics,12/5/2020,https://www.dawnnews.tv/news/1148430/,Dawn News,2349.0
4,4,کے الیکٹرک کو اضافی بجلی گیس کی فراہمی کے قانو...,اسلام باد نیشنل ٹرانسمیشن اینڈ ڈسپیچ کمپنی این...,Business & Economics,12/5/2020,https://www.dawnnews.tv/news/1148421/,Dawn News,2655.0


##Preprocessing of the Dataset

We will remove some of the extra columns that we are not going to use 

In [ ]:
data_pre=df.drop(columns=['Index','Date','URL','Source','News length'])
data_pre.head()
data_pre=data_pre.loc[df['Category'] == 'Business & Economics']

In [ ]:
data_pre['News Text'] = \
data_pre['News Text'].map(lambda x: re.sub('[,\\.!?]', '', str(x)))
data_pre['News Text'].head()

0    اسلام باد عالمی بینک خیبرپختونخوا کے قبائلی اض...
1    اسلام باد فیڈرل بورڈ ریونیو ایف بی نے دسمبر کی...
2    اسلام باد بورڈ انویسٹمنٹ بی او ئی کے چیئرمین ع...
3    اسلام اباد پاکستان میں ماہ نومبر میں مسلسل تیس...
4    اسلام باد نیشنل ٹرانسمیشن اینڈ ڈسپیچ کمپنی این...
Name: News Text, dtype: object

In [ ]:
import nltk
stopwords=pd.read_csv('/content/gdrive/MyDrive/Dataset(Colab)/UrduTopicModelling/stopwords-ur.txt',names=['List'])
# stopwords['List']

stopwords_ur=[]
for li in stopwords['List']:
  stopwords_ur.append(li)
stopwords_ur.extend(['میں','سے','کا','نے','کو','بھی'])
print(stopwords_ur)

['آئی', 'آئے', 'آج', 'آخر', 'آخرکبر', 'آدهی', 'آًب', 'آٹھ', 'آیب', 'اة', 'اخبزت', 'اختتبم', 'ادھر', 'ارد', 'اردگرد', 'ارکبى', 'اش', 'اضتعوبل', 'اضتعوبلات', 'اضطرذ', 'اضکب', 'اضکی', 'اضکے', 'اطراف', 'اغیب', 'افراد', 'الگ', 'اور', 'اوًچب', 'اوًچبئی', 'اوًچی', 'اوًچے', 'اى', 'اً', 'اًذر', 'اًہیں', 'اٹھبًب', 'اپٌب', 'اپٌے', 'اچھب', 'اچھی', 'اچھے', 'اکثر', 'اکٹھب', 'اکٹھی', 'اکٹھے', 'اکیلا', 'اکیلی', 'اکیلے', 'اگرچہ', 'اہن', 'ایطے', 'ایک', 'ب', 'ت', 'تبزٍ', 'تت', 'تر', 'ترتیت', 'تریي', 'تعذاد', 'تن', 'تو', 'توبم', 'توہی', 'توہیں', 'تٌہب', 'تک', 'تھب', 'تھوڑا', 'تھوڑی', 'تھوڑے', 'تھی', 'تھے', 'تیي', 'ثب', 'ثبئیں', 'ثبترتیت', 'ثبری', 'ثبرے', 'ثبعث', 'ثبلا', 'ثبلترتیت', 'ثبہر', 'ثدبئے', 'ثرآں', 'ثراں', 'ثرش', 'ثعذ', 'ثغیر', 'ثلٌذ', 'ثلٌذوثبلا', 'ثلکہ', 'ثي', 'ثٌب', 'ثٌبرہب', 'ثٌبرہی', 'ثٌبرہے', 'ثٌبًب', 'ثٌذ', 'ثٌذکرو', 'ثٌذکرًب', 'ثٌذی', 'ثڑا', 'ثڑوں', 'ثڑی', 'ثڑے', 'ثھر', 'ثھرا', 'ثھراہوا', 'ثھرپور', 'ثھی', 'ثہت', 'ثہتر', 'ثہتری', 'ثہتریي', 'ثیچ', 'ج', 'خب', 'خبرہب', 'خبرہی', 'خبرہے', 'خبهوظ

In [ ]:
import gensim
from gensim.utils import simple_preprocess
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

In [ ]:
data = data_pre['News Text'].values.tolist()
print((data[19]))

ممبئی بھارت کی معیشت جولائی اور ستمبر کے دوران 75 فیصد سکڑنے سے بدترین کارکردگی کا مظاہرہ کرنے والی بڑی ترقی یافتہ اور ابھرتی ہوئی معیشتوں میں شامل ہوگئی کیونکہ یہ ازادی کے بعد پہلی مرتبہ تکنیکی کساد بازاری میں داخل ہوئی ہےڈان اخبار میں شائع فرانسیسی خبررساں ادارے اے ایف پی کی رپورٹ میں بتایا گیا کہ سرکاری اعداد شمار ظاہر کرتے ہیں کہ معیشت کساد بازاری میں داخل ہوگئی ہےاگرچہ گزشتہ سہ ماہی میں ریکارڈ 239 فیصد سکڑنے کے مقابلے میں اعداد شمار میں بہتری تھی تاہم یہ اس طرف اشارہ کرتے ہیں کہ ایشیا کی تیسری بڑی معیشت سخت مقابلہ کر رہی ہے کیونکہ یہ طلب کو بحال کرنے اور روزگار پیدا کرنے کی کوششوں میں ہے جبکہ کورونا وائرس کا انفیکشن بڑھ رہا ہےمزید پڑھیں بھارت میں کورونا کی ابتر صورتحال ایک دن میں 90 ہزار سے زائد کیسز رپورٹتاہم مسلسل سہ ماہیوں میں معیشت کے سکڑنے کا مطلب ہے کہ ملک 1947 کے بعد سے پہلی مرتبہ تکنیکی کساد بزاری میں داخل ہوگیا ہےوائرس سے متعلق لاک ڈانز سے ہونے والی عالمی تباہی کے بعد امریکا جاپان اور جرمنی سمیت بڑی معیشتوں کی جانب سے 30 ستمبر کو ختم ہونے والی سہ ماہی میں ریکارڈ کی گئی تر

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

In [ ]:
vectorizer=TfidfVectorizer(stop_words=stopwords_ur,use_idf=True,encoding='utf-8')

X=vectorizer.fit_transform(data)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['آش', 'ئی', 'ئے', 'او', 'اٹھب', 'اپ', 'بة', 'بت', 'برہب', 'برہی', 'برہے', 'بضت', 'بپط', 'بگسیر', 'تب', 'تبز', 'تی', 'تے', 'ثل', 'دکھب', 'دیکھ', 'ذر', 'ذوثبلا', 'ذکر', 'ذکرو', 'ذی', 'زیبد', 'ضبد', 'ضک', 'ضیک', 'طب', 'طجت', 'طی', 'طے', 'علاو', 'قطہ', 'لگ', 'هوک', 'هڑ', 'وخواى', 'وں', 'پب', 'پرا', 'پط', 'پوچھ', 'چب', 'چبئی', 'چبہ', 'چی', 'چے', 'ڈا', 'ڈلیب', 'ڈو', 'ڈھو', 'ڈی', 'ڈیں', 'کبل', 'کتہ', 'کر', 'کور', 'کھول', 'کیو', 'گرو', 'ہب', 'یب', 'یقی'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [ ]:
print(X.shape)
print(X[0])

(24130, 110797)
  (0, 56337)	0.04411514863716309
  (0, 35373)	0.048804087478820035
  (0, 26758)	0.0546623769929535
  (0, 73006)	0.026956161823694404
  (0, 40724)	0.05012116171917914
  (0, 65034)	0.03935527992009309
  (0, 18352)	0.035753811092527704
  (0, 90319)	0.050782409076755575
  (0, 56984)	0.06719368438140569
  (0, 80919)	0.06777286252021084
  (0, 22504)	0.06811112217053714
  (0, 55867)	0.05400301500253182
  (0, 108356)	0.054245021292081246
  (0, 46533)	0.05404292980444918
  (0, 47062)	0.04211095024085394
  (0, 29887)	0.03721477189311603
  (0, 64890)	0.04096399836569487
  (0, 53059)	0.09138868831180619
  (0, 31358)	0.04267216420863056
  (0, 22132)	0.06957254765639305
  (0, 37989)	0.03726375630069041
  (0, 93615)	0.016225939706425226
  (0, 57011)	0.05161596361903408
  (0, 49044)	0.08147418313158808
  (0, 55995)	0.06364533079073487
  :	:
  (0, 58546)	0.044346389972313466
  (0, 62742)	0.06165568261275348
  (0, 65647)	0.047281734317677765
  (0, 34169)	0.04752273214241591
  (0, 64517)	

In [ ]:
lsa=TruncatedSVD(n_components=68, n_iter=100)
lsa.fit(X)

TruncatedSVD(algorithm='randomized', n_components=68, n_iter=100,
             random_state=None, tol=0.0)

In [ ]:
print(len(lsa.components_))
print(len(lsa.components_[0])) #lsa.components_ has 68 element and each element has further 214488 elements in it

# import numpy as np
# num_array=np.array(lsa.components_)
# print(np.sum(num_array))

68
110797


In [ ]:
terms=vectorizer.get_feature_names() #it extracts all the actual words corresponding to the id
# print(len(terms))
for i,comp in enumerate(lsa.components_):
  termsInComp=zip(terms,comp) #map the 2 arrays(one array with TF-IDF Scores and other with corresponding word)
  sortedTerms=sorted(termsInComp, key=lambda x:x[1], reverse=True) [:10] #sorting to get maximum TF-IDF first and only taking first 10 values of it
  print("Concept:",i)
  for term in sortedTerms:
    print(term[0])
  print(" ")
  if i==5:
    break


Concept: 0
روپے
ڈالر
قیمت
ارب
کمی
مارکیٹ
پاکستان
فیصد
پیسے
ہزار
 
Concept: 1
سونے
مارکیٹ
قیمت
روپے
پوائنٹس
پیسے
انڈیکس
ڈالر
تولہ
ہزار
 
Concept: 2
قیمت
روپے
جی
این
سونے
پیسے
گیس
سی
لیٹر
قیمتوں
 
Concept: 3
این
جی
سی
گیس
پوائنٹس
انڈیکس
اسٹیشنز
بند
اسٹاک
صبح
 
Concept: 4
لیٹر
ڈیزل
پیسے
مصنوعات
قیمتوں
پوائنٹس
پیٹرولیم
انڈیکس
پیٹرول
ٹیکس
 
Concept: 5
سونے
ٹیکس
تولہ
ایف
صرافہ
گرام
ہزار
بی
اونس
سونا
 
